# 최대 연속 부분합을 구하는 4가지 알고리즘

Codility의 MaxSliceSum 문제. 정수 배열에서!

Largest sum of continuous subarray in a non-empty array

## 1. 완전탐색 : O(2<sup>n</sup>)

일단 다음과 같은 부분문제를 만들기

f(now, tmp_ans) : 현재 탐색할 인덱스가 now이고, 그전까지 만든 최대값이 tmp_ans일때, 배열의 끝에서의 최대 연속된 부분수열의 합
고려해야 할 경우의 수는 다음 4가지

1. 배열의 현재 인덱스값 하나만 가지고 끝나는경우 

2. 지금까지 구해온 답에 현재 배열의 now 인덱스 값을 더해서 끝내는 경우

3. 지금까지 만든 값을 버리고 현재 인덱스 값을 가지고 다시 시작하는 경우

4. 현재 값과 지금까지 만든 값을 포함해서 계속해서 답을 찾아나가는 경우

현재 인덱스에서 답을 찾을 수 있는 모든 경우의 수는 이 4가지 뿐이며 중복되거나 놓친 부분이 없다.

결국 이 4가지 값중 최대의 값을 찾으면 그 인덱스에서의 최대값이 된다.

In [ ]:
MIN = -2 ** 31 - 1

def exhaustive_search(arr):
    N = len(arr)
    
    def find(now, tmp_ans):
        if now == N:
            return MIN
        
        ans = max(arr[now], #1.
                  tmp_ans + arr[now], #2
                  find(now + 1, arr[now]), #3
                  find(now + 1, tmp_ans + arr[now]) #4
                  )
        return ans
    return find(0, 0)

## 2. 부분합 수열 : O(n<sup>2</sup>)

  부분합 수열은 인덱스 i의 값에 수열 arr의 0부터 i 인덱스까지의 합을 담은 수열


In [ ]:
def partial_sum(arr):
    # 1.
    arr = [0] + arr
    N = len(arr)
    p_sum = [0] * N
    ans = MIN
    # 2.
    for i in range(1, N):
        p_sum[i] = p_sum[i-1] + arr[i]
    # 3.
    for hi in range(1, N):
        for lo in range(1, hi+1):
            ans = max(ans, p_sum[hi] - p_sum[lo-1])
    return ans

## 3. 분할정복 : `O(nlog2n)`

 우리가 정답을 구할 인덱스의 최소값을 lo, 최대값을 hi라고 하자. 그리고 그 중간 인덱스는 mid이다. 이때 정답을 만드는 구간은 세가지 중 하나이다.
 
 1. [lo,mid] 원 배열의 왼쪽 절반
 2. [mid+1, hi] 원 배열의 오른쪽 절반
 3. 그 외, 양쪽 절반에 걸친 경우
 
**위 세가지 중 반드시 정답이 있으며 놓친 부분은 없다.**
 

In [ ]:
def divide_conquer(arr):
    N = len(arr)
    def find(lo, hi):
        #1.
        if lo == hi:
            return arr[lo]
        mid = (lo + hi) // 2
        #2.
        left = find(lo, mid)
        right = find(mid+1, hi)
        
        #3.
        tmp = 0
        left_part = MIN
        for i in range(mid, lo-1, -1):
            tmp += arr[i]
            left_part = max(left_part, tmp)
            
        tmp = 0
        right_part = MIN
        for i in range(mid+1, hi+1):
            tmp += arr[i]
            right_part = max(right_part, tmp)
            
        #4.
        return max(left, right, left_part + right_part)
    #5.
    return find(0, N-1)

1. 분할과 정복을 동시에 수행하는 find 함수를 정의한다.
2. 두 경우의 수를 구한다.
3. 마지막으로 최대합이 두 구간에 걸쳐 있을때의 답을 구한다.
4. 정답은 세 가지 경우의 수의 최대값이다.
5. 함수를 시작한다. 원 배열의 시작과 끝 인덱스를 넣고 답을 찾아나간다.

시간복잡도는 배열을 절반씩 나누기 때문에 기본적으로 log2n인데 이렇게 나눌때마다 n씩 답을 찾기 떄문에 nlog2n

---
## 4. 동적 계획법 : O(n)

 가장 정석적이고 바람직하며 간단한 방법
 
 어떤 문제에 대해 해결가능한 더 작은 부분문제로 나눌 수 있다면 해결 뿐 아니라 최적화가 가능해지는데, 이 문제에 대해 다음과 같은 부분문제를 만들 수 있다고 한다.
 
 > 배열 arr에 대해서, f(i) = 인덱스 i를 오른쪽 끝으로 갖는 구간으 최대합
    
 이 때 다음과 같은 식이 성립
```
 f(i) = max(0, f(i-1)) + arr[i] when i > 0,
        arr[i]                       i == 0   
```

In [ ]:
def dynamic_programming(arr):
    cache = [None] * len(arr)
    # 1.
    cache[0] = arr[0]
    
    # 2.
    for i in range(1, len(arr)):
        cache[i] = max(0, cache[i-1]) + arr[i]
    
    return max(cache)